In [2]:
# Import libraries

import torch
import torch.nn as nn
import torch.optim as optim


In [3]:
# Defining the multi task learning model

class MultiTaskModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MultiTaskModel, self).__init__()
        self.shared_layer = nn.Linear(input_size, hidden_size)
        self.age_layer = nn.Linear(hidden_size, 1)
        self.dogs_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        shared_output = torch.relu(self.shared_layer(x))
        age_output = self.age_layer(shared_output)
        dogs_output = self.dogs_layer(shared_output)
        return age_output, dogs_output


In [4]:
# Defining personalized loss function

class WeightedLoss(nn.Module):
    def __init__(self, weight_age, weight_dogs):
        super(WeightedLoss, self).__init__()
        self.weight_age = weight_age
        self.weight_dogs = weight_dogs
        self.mse_loss = nn.MSELoss()
    
    def forward(self, age_pred, age_true, dogs_pred, dogs_true):
        loss_age = self.mse_loss(age_pred, age_true)
        loss_dogs = self.mse_loss(dogs_pred, dogs_true)
        return self.weight_age * loss_age + self.weight_dogs * loss_dogs


In [5]:
# Initialize the model, loss function and optimizer

input_size = 2    # Numero di caratteristiche di input (età e numero di cani)
hidden_size = 5   # Numero di unità nascoste

model = MultiTaskModel(input_size, hidden_size)

loss_fn = WeightedLoss(weight_age=90, weight_dogs=10)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
# Creating a dataset

# I nostri dati: [età, numero di cani]
data = torch.tensor([
    [10, 5],  # Martina
    [14, 4],  # Luigi
    [40, 8]   # Maria
], dtype=torch.float32)

# Target: vogliamo minimizzare età e numero di cani
target_age = torch.tensor([
    [10],
    [14],
    [40]
], dtype=torch.float32)

target_dogs = torch.tensor([
    [5],
    [4],
    [8]
], dtype=torch.float32)


In [7]:
# Training process

num_epochs = 100

for epoch in range(num_epochs):
    age_output, dogs_output = model(data)
    loss = loss_fn(age_output, target_age, dogs_output, target_dogs)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete!")


Epoch [10/100], Loss: 45800.6914
Epoch [20/100], Loss: 44710.1445
Epoch [30/100], Loss: 43660.8398
Epoch [40/100], Loss: 42640.6875
Epoch [50/100], Loss: 41636.2031
Epoch [60/100], Loss: 40635.8008
Epoch [70/100], Loss: 39638.0742
Epoch [80/100], Loss: 38659.3516
Epoch [90/100], Loss: 37661.2461
Epoch [100/100], Loss: 36644.4219
Training complete!


In [8]:
# Use the model to select the person

model.eval()
with torch.no_grad():
    age_pred, dogs_pred = model(data)
    scores = 90 * age_pred + 10 * dogs_pred
    min_index = torch.argmin(scores).item()
    selected_person = ["Martina", "Luigi", "Maria"][min_index]

print(f"The selected person with the minimum weighted score is: {selected_person}")


The selected person with the minimum weighted score is: Martina
